# IMPORTING MODULES

In [104]:
%config Completer.use_jedi = False

In [1]:
from arabic_reshaper import arabic_reshaper
from bidi.algorithm import get_display
from time import sleep
import pandas as pd
import regex as re
from itertools import chain
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
akbar_colors = ['#009933',
 '#006699',
 '#333333',
 '#669999',
 '#990066',
 '#996666',
 '#cc3333',
 '#cc6633',
 '#cc6666',
 '#ff6633',
 '#990000',
 '#003366',
 '#339966',
 '#cc3399',
 '#3333ff']
my_cmap = matplotlib.colors.ListedColormap(akbar_colors, name='my_colormap_name', N=45)

In [6]:
pickle_text = 'ALL_TEXT.pkl' # WORK ON THIS FIRST [TODO]

In [7]:
df01 = pd.read_pickle(pickle_text)

In [9]:
df01['LENGTH'] = df01['TEXT'].apply(lambda x: len(x.split()))

In [11]:
df01['NLP'] =  df01['TEXT'].apply(lambda x: nlp(x))

In [15]:
df01['Noun_Phrases'.upper()] = df01['NLP'].apply(lambda x: [chunk.text for chunk in x.noun_chunks])

In [16]:
df01['Verbs'] = df01['NLP'].apply(lambda x: [token.lemma_ for token in x if token.pos_ == "VERB"])
df01['ADJ'] = df01['NLP'].apply(lambda x: [token.lemma_ for token in x if token.pos_ == "ADJ"])
df01['NOUN'] = df01['NLP'].apply(lambda x: [token.lemma_ for token in x if token.pos_ == "NOUN"])
df01['PROPN'] = df01['NLP'].apply(lambda x: [token.lemma_ for token in x if token.pos_ == "PROPN"])
df01['ADV'] = df01['NLP'].apply(lambda x: [token.lemma_ for token in x if token.pos_ == "ADV"])

In [20]:
col = 'TEXT	LENGTH		NOUN_PHRASES	Verbs	ADJ	NOUN	PROPN	ADV'
col = col.split()
df01 = df01[col]

In [21]:
df01

,TEXT,LENGTH,NOUN_PHRASES,Verbs,ADJ,NOUN,PROPN,ADV
0,About Membership | The GIIN,5,"[Membership |, The GIIN]",[],[],[],"[Membership, |, GIIN]",[]
1,Find It,2,[It],[find],[],[],[],[]
2,Membership,1,[Membership],[],[],[membership],[],[]
3,Login,1,[Login],[],[],[],[Login],[]
4,About Impact Investing,3,[],[invest],[],[],[Impact],[]
...,...,...,...,...,...,...,...,...
70592,IMPACT INVESTOR TYPES AND LOCAL PRESENCE IN WE...,121,"[IMPACT INVESTOR TYPES, LOCAL PRESENCE, WEST A...","[•, •, d’Ivoire, note, will, equal]","[local, local, -, institutional, local, local,...","[type, local, presence, west, investor, presen...","[IMPACT, INVESTOR, AFRICA, JULY, Impact, Inves...",[]
70593,"Foundations, relying on capital from sources s...",17,"[Foundations, capital, sources, high-net-worth...",[rely],"[such, high, worth]","[foundation, capital, source, net, individual]","[HWNIs, West, African, Development, Bank]",[]
70594,• Benin • Burkina Faso • Cote d’Ivoire • Guine...,51,"[•, Benin, Burkina Faso, Cote d’Ivoire, •, Gui...","[•, •, •, compare, deploy, compare]","[more, non, -, waemu]","[d’Ivoire, impact, investment, activity, count...","[•, Benin, Burkina, Faso, Cote, •, Guinea, Bis...",[significantly]
70595,"ECOSYSTEM ACTORS IN WEST AFRICA, JULY 2015\n\n...",140,"[ECOSYSTEM ACTORS, WEST AFRICA, JULY, Innovate...",[],[],"[et, croissance, l5lab, idea, et, association,...","[ECOSYSTEM, ACTORS, WEST, AFRICA, JULY, Innova...","[accompagnateur, efina]"


# Titling Function for the Noun Phrases

In [22]:
def proper_case(text):
    '''Given a string, convert its words to the Title form, unless the word is ACRONYM!'''
    pc = " ".join([i.title() if not i.isupper() else i for i in text.split()])
    return pc

In [23]:
stop_words = "how very nbsp will years year be other only often first how him rather another but he have many three near yours won us to we'll usually ours haven't their couldn't together thing please with through an is been ain within since vi one do much your she's after all iii is't self for could away done thus always some also doing will where and does iv aren wasn than well about a truth till even mrs just get are therefore little didn't yet until me who won't they almost far doesn't in we because third else once enough himself father's such be from while should things when too weren't thoughts speed you've not hadn't less on between without was ii why them here's i ha never v that's yourself each last his being say age any shouldn't if make shall had themselves nor there which herself said her though half it's i'll these viii needn't by so out same were didn am ll myself my give hour now shouldn she you'd way ve vii still keep take no here mustn't cannot indeed our as you'll having under couldn gone you're can yes at theirs i' before long let's whose hers the two nothing few may over whom what wish isn't might des there's again ix mightn't should've hadn given wrong come yourselves shan old wasn't own he's sit this its thousand seen that'll o must what's hasn enter very alone below wouldn aren't x tell it mine love more during don't name those wouldn't hasn't others has every forth then itself further against would into both none mightn did most ourselves or you back ever that"
stop_words = stop_words.split()
stop_words = set(stop_words)

In [24]:
print(f'English Stopwords Count: {len(stop_words)}')

English Stopwords Count: 273


In [105]:
np_list = list(df01['NOUN_PHRASES'])

In [106]:
np_list = list(chain(*np_list))

In [107]:
np_list = [proper_case(i) for i in np_list]

In [108]:
noun_phrases_freq = Counter(np_list)

In [109]:
noun_phrases_freq  = dict(noun_phrases_freq)

In [110]:
df_phrases = pd.DataFrame.from_dict(noun_phrases_freq, orient='index').reset_index()

In [111]:
df_phrases = df_phrases.rename(columns={'index':'Phrase', 0:'Freq'})

In [112]:
df_phrases = df_phrases.sort_values('Freq', ascending=False)

# Get rid of stopwords

In [113]:
df_phrases['Phrase_sw'] = df_phrases['Phrase'].apply(lambda x: " ".join([i if i.lower() not in stop_words else "" for i in x.split()]))

In [114]:
df_phrases['Length'] = df_phrases['Phrase_sw'].apply(lambda x: len(x.split()))
df_phrases = df_phrases[df_phrases['Length']>1]

In [115]:
df_phrases =  df_phrases['Freq	Phrase_sw'.split()]

In [116]:
df_phrases['Phrase_sw_new'] = df_phrases['Phrase_sw'].apply(lambda x: x.title())

In [117]:
df_phrases = df_phrases.drop_duplicates(['Phrase_sw_new'])
df_phrases = df_phrases.reset_index(drop=True)

In [118]:
df_phrases = df_phrases[df_phrases['Freq']>2]
df_phrases = df_phrases.reset_index(drop=True)

In [121]:
df_phrases = df_phrases['Phrase_sw_new Freq'.split()]
df_phrases = df_phrases.rename(columns={'Phrase_sw_new':'TERM', 'Freq':'FREQ'})

In [122]:
terms_dict = df_phrases.head(1000)

In [123]:
terms_dict  = dict(df_phrases.values.tolist())

In [124]:
wordcloud_terms = WordCloud(width=1920*2, height=1080*2, background_color="#F4ECF7", colormap=my_cmap, font_path='./font/Roboto-Regular.ttf').generate_from_frequencies(terms_dict)
wordcloud_terms.to_file('TERMS.png')

In [125]:
df_phrases.to_excel('TERMS.xlsx', encoding='utf-8', index=False)

In [126]:
del terms_dict, df_phrases, noun_phrases_freq, np_list

In [127]:
df01

,TEXT,LENGTH,NOUN_PHRASES,Verbs,ADJ,NOUN,PROPN,ADV
0,About Membership | The GIIN,5,"[Membership |, The GIIN]",[],[],[],"[Membership, |, GIIN]",[]
1,Find It,2,[It],[find],[],[],[],[]
2,Membership,1,[Membership],[],[],[membership],[],[]
3,Login,1,[Login],[],[],[],[Login],[]
4,About Impact Investing,3,[],[invest],[],[],[Impact],[]
...,...,...,...,...,...,...,...,...
70592,IMPACT INVESTOR TYPES AND LOCAL PRESENCE IN WE...,121,"[IMPACT INVESTOR TYPES, LOCAL PRESENCE, WEST A...","[•, •, d’Ivoire, note, will, equal]","[local, local, -, institutional, local, local,...","[type, local, presence, west, investor, presen...","[IMPACT, INVESTOR, AFRICA, JULY, Impact, Inves...",[]
70593,"Foundations, relying on capital from sources s...",17,"[Foundations, capital, sources, high-net-worth...",[rely],"[such, high, worth]","[foundation, capital, source, net, individual]","[HWNIs, West, African, Development, Bank]",[]
70594,• Benin • Burkina Faso • Cote d’Ivoire • Guine...,51,"[•, Benin, Burkina Faso, Cote d’Ivoire, •, Gui...","[•, •, •, compare, deploy, compare]","[more, non, -, waemu]","[d’Ivoire, impact, investment, activity, count...","[•, Benin, Burkina, Faso, Cote, •, Guinea, Bis...",[significantly]
70595,"ECOSYSTEM ACTORS IN WEST AFRICA, JULY 2015\n\n...",140,"[ECOSYSTEM ACTORS, WEST AFRICA, JULY, Innovate...",[],[],"[et, croissance, l5lab, idea, et, association,...","[ECOSYSTEM, ACTORS, WEST, AFRICA, JULY, Innova...","[accompagnateur, efina]"


In [128]:
verbs_list = list (df01['Verbs'])

In [129]:
verbs_list = list(chain(*verbs_list))

In [130]:
verbs_list = [i for i in verbs_list if i.lower() not in stop_words]

In [131]:
verb_freq = Counter(verbs_list)

In [132]:
verb_freq  = dict(verb_freq)

In [133]:
df_verbs = pd.DataFrame.from_dict(verb_freq, orient='index').reset_index()

In [134]:
df_verbs = df_verbs.rename(columns={'index':'Verb', 0:'Freq'})

In [135]:
df_verbs = df_verbs.sort_values('Freq', ascending=False).reset_index(drop=True)

In [136]:
df_verbs = df_verbs[df_verbs['Freq']>2]

In [138]:
df_verbs = df_verbs[df_verbs['Verb'].apply(lambda x: x.isalpha())].reset_index(drop=True)

In [139]:
df_verbs.to_excel('VERBS.xlsx', encoding='utf-8', index=False)

In [140]:
verb_dict = {k:v for k,v in zip(df_verbs['Verb'], df_verbs['Freq'])}

In [141]:
wordcloud_verbs = WordCloud(width=1920*2, height=1080*2, background_color="#F5EEF8", colormap=my_cmap, font_path='font/Roboto-Regular.ttf').generate_from_frequencies(verb_dict)

In [142]:
wordcloud_verbs.to_file('VERBS.png')

In [174]:
del df_verbs, verb_dict, verb_freq, verbs_list

In [143]:
noun_list = list (df01['NOUN'])

In [144]:
noun_list = list(chain(*noun_list))

In [145]:
noun_list = [i for i in noun_list if i.lower() not in stop_words]

In [146]:
noun_freq = Counter(noun_list)

In [147]:
noun_freq  = dict(noun_freq)

In [148]:
df_noun = pd.DataFrame.from_dict(noun_freq, orient='index').reset_index()

In [149]:
df_noun = df_noun.rename(columns={'index':'Noun', 0:'Freq'})

In [150]:
df_noun = df_noun.sort_values('Freq', ascending=False).reset_index(drop=True)

In [151]:
df_noun = df_noun[df_noun['Freq']>2]

In [152]:
df_noun.to_excel('NOUNS.xlsx', encoding='utf-8', index=False)

In [153]:
noun_dict = {k:v for k,v in zip(df_noun['Noun'] , df_noun['Freq'])}

In [154]:
noun_dict = {k:v for k,v in noun_dict.items() if k.isalpha()}

In [155]:
wordcloud_noun = WordCloud(width=1920*2, height=1080*2, background_color="#E8F8F5", colormap=my_cmap, font_path='font/Roboto-Regular.ttf').generate_from_frequencies(noun_dict)

In [157]:
wordcloud_noun.to_file('NOUN.png')

In [173]:
del noun_dict, noun_freq, noun_list, df_noun

In [158]:
pnoun_list = list (df01['PROPN'])

In [159]:
pnoun_list = list(chain(*pnoun_list))

In [160]:
pnoun_list = [i for i in pnoun_list if i.lower() not in stop_words]

In [161]:
pnoun_freq = Counter(pnoun_list)

In [162]:
pnoun_freq  = dict(pnoun_freq)

In [163]:
df_pnoun = pd.DataFrame.from_dict(pnoun_freq, orient='index').reset_index()

In [164]:
df_pnoun = df_pnoun.rename(columns={'index':'Proper_Noun', 0:'Freq'})

In [165]:
df_pnoun = df_pnoun.sort_values('Freq', ascending=False).reset_index().drop(columns=['index'])

In [166]:
df_pnoun = df_pnoun[df_pnoun['Freq']>2]

In [167]:
df_pnoun = df_pnoun[df_pnoun['Proper_Noun'].apply(lambda x: x not in [',', '.', '_', '×', 'Q', 'et', 'al', '=', '|', '\\', '#'])]

In [168]:
df_pnoun.to_excel('PROPER_NOUNS.xlsx', encoding='utf-8', index=False)

In [169]:
pnoun_dict = {k:v for k,v in zip(df_pnoun['Proper_Noun'] , df_pnoun['Freq'])}

In [170]:
pnoun_dict = {k:v for k,v in pnoun_dict.items() if k.isalpha()}

In [171]:
pnoun_dict

{'Impact': 6052,
 'GIIN': 4828,
 'IMPACT': 2952,
 'Development': 2891,
 'Investment': 2592,
 'Fund': 2541,
 'Africa': 2495,
 'Capital': 2266,
 'Investing': 2083,
 'Foundation': 1905,
 'Global': 1870,
 'Bank': 1850,
 'Social': 1749,
 'IRIS': 1626,
 'INVESTING': 1554,
 'de': 1521,
 'World': 1401,
 'Network': 1383,
 'Finance': 1372,
 'South': 1224,
 'Figure': 1160,
 'Management': 1133,
 'DFI': 1103,
 'FIGURE': 1097,
 'Asia': 1096,
 'Investors': 1064,
 'International': 1036,
 'AUM': 1029,
 'United': 989,
 'Group': 978,
 'Research': 963,
 'AFRICA': 938,
 'India': 887,
 'la': 884,
 'Financial': 881,
 'Ghana': 833,
 'USD': 832,
 'à': 827,
 'Investor': 805,
 'les': 797,
 'East': 784,
 'New': 775,
 'Business': 767,
 'Community': 732,
 'IMM': 728,
 'Private': 726,
 'Investments': 714,
 'Economic': 703,
 'GDP': 693,
 'ESG': 692,
 'Services': 691,
 'Report': 678,
 'Morgan': 650,
 'le': 649,
 'Survey': 648,
 'Nigeria': 591,
 'African': 588,
 'Market': 548,
 'Energy': 547,
 'Funds': 542,
 'Performan

In [172]:
wordcloud_pnoun = WordCloud( width=1920*2, height=1080*2, background_color="#F9EBEA", colormap=my_cmap, font_path = 'font/Roboto-Regular.ttf').generate_from_frequencies(pnoun_dict)
wordcloud_pnoun.to_file('PROPER_NOUNS.png')

In [175]:
del pnoun_dict, pnoun_freq, pnoun_list, df_pnoun

In [176]:
adj_list = list (df01['ADJ'])

In [177]:
adj_list = list(chain(*adj_list))

In [178]:
adj_list = [i for i in adj_list if i.lower() not in stop_words]

In [179]:
adj_freq = Counter(adj_list)

In [180]:
adj_freq  = dict(adj_freq)

In [181]:
df_adj = pd.DataFrame.from_dict(adj_freq, orient='index').reset_index()

In [182]:
df_adj = df_adj.rename(columns={'index':'Adjective', 0:'Freq'})

In [183]:
df_adj = df_adj.sort_values('Freq', ascending=False).reset_index().drop(columns=['index'])

In [184]:
df_adj = df_adj[df_adj['Freq']>2]

In [185]:
df_adj = df_adj[df_adj['Adjective'].apply(lambda x: x not in [',', '.', '_', '×', 'Q', 'et', 'al', '='])]

In [187]:
df_adj.to_excel('ADJECTIVES.xlsx', encoding='utf-8', index=False)

In [188]:
adjective_dict = {k:v for k,v in zip(df_adj.Adjective , df_adj.Freq )}

In [189]:
adjective_dict = {k:v for k,v in adjective_dict.items() if k.isalpha()}

In [190]:
wordcloud_adj = WordCloud( width=1920*2, height=1080*2, background_color="#EBF5FB", colormap=my_cmap, font_path='font/Roboto-Regular.ttf').generate_from_frequencies(adjective_dict)

In [192]:
wordcloud_adj.to_file('ADJECTIVES.png')

In [193]:
del adj_freq, adj_list, adjective_dict, df_adj

In [194]:
adv_list = list (df01['ADV'])

In [195]:
adv_list = list(chain(*adv_list))

In [196]:
adv_list = [i for i in adv_list if i.lower() not in stop_words]

In [197]:
adv_freq = Counter(adv_list)

In [198]:
adv_freq  = dict(adv_freq)

In [199]:
df_adv = pd.DataFrame.from_dict(adv_freq, orient='index').reset_index()

In [200]:
df_adv = df_adv.rename(columns={'index':'Adverb', 0:'Freq'})

In [201]:
df_adv = df_adv.sort_values('Freq', ascending=False).reset_index().drop(columns=['index'])

In [202]:
df_adv = df_adv[df_adv['Freq']>2]

In [203]:
df_adv = df_adv[df_adv['Adverb'].apply(lambda x: x not in [',', '.', '_', '×', 'Q', 'et', 'al', '='])]

In [204]:
df_adv.to_excel('ADVERBS.xlsx', encoding='utf-8', index=False)

In [205]:
adverb_dict = {k:v for k,v in zip(df_adv['Adverb'] , df_adv['Freq'])}

In [206]:
adverb_dict = {k:v for k,v in adverb_dict.items() if k.isalpha()}

In [208]:
adverb_dict

{'however': 1355,
 'particularly': 710,
 'nearly': 607,
 'primarily': 584,
 'currently': 567,
 'typically': 562,
 'directly': 529,
 'especially': 527,
 'relatively': 522,
 'generally': 514,
 'already': 385,
 'approximately': 378,
 'significantly': 370,
 'increasingly': 369,
 'specifically': 320,
 'commonly': 307,
 'largely': 298,
 'globally': 280,
 'additionally': 279,
 'respectively': 265,
 'recently': 259,
 'least': 247,
 'forward': 235,
 'effectively': 234,
 'publicly': 233,
 'overall': 231,
 'fully': 229,
 'highly': 226,
 'above': 222,
 'actively': 222,
 'otherwise': 215,
 'notably': 203,
 'previously': 200,
 'socially': 199,
 'similarly': 195,
 'prior': 192,
 'close': 188,
 'ultimately': 181,
 'widely': 180,
 'second': 174,
 'potentially': 165,
 'somewhat': 164,
 'annually': 163,
 'likely': 162,
 'roughly': 159,
 'mostly': 157,
 'financially': 157,
 'finally': 157,
 'instead': 156,
 'slightly': 156,
 'broadly': 152,
 'environmentally': 152,
 'around': 151,
 'necessarily': 150,
 'f

In [209]:
wordcloud_adv = WordCloud( width=1920*2, height=1080*2, background_color="#EBF5FB", colormap=my_cmap, font_path='font/Roboto-Regular.ttf').generate_from_frequencies(adverb_dict)

In [211]:
wordcloud_adv.to_file('ADVERBS.png')

In [212]:
del df_adv, adv_freq, adv_list, adverb_dict